## Load flow computation in the IEEE 9-bus test system

The purpose of this notebook is to show how pan can be used, via its Python interface, to compute the load flow of a power network

The example network used is the IEEE 9-bus test system shown below:

<a href="https://www.researchgate.net/figure/Diagram-of-the-IEEE-9-bus-test-system_fig2_303381482"><img src="https://www.researchgate.net/profile/Yue-Song-4/publication/303381482/figure/fig2/AS:368524460609538@1464874486665/Diagram-of-the-IEEE-9-bus-test-system.png" alt="Diagram of the IEEE 9-bus test system."/></a>

This network contains three generators, three loads and nine buses.

In [ ]:
import pypan.ui as pan
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
import pickle
%matplotlib inline

#### Default active and reacitve powers of the loads and voltage ratings of the buses

In [ ]:
LOADS = {
    'A': {'P': 125., 'Q': 50.},
    'B': {'P':  90., 'Q': 30.},
    'C': {'P': 100., 'Q': 35.}
}

VRATING = {
    'bus1': 16.5,
    'bus2': 18,
    'bus3': 13.8,
}
for bus in range(4,10):
    VRATING[f'bus{bus}'] = 230

This function converts the data returned by pan to the same format as saved in PowerFactory, for ease of comparison.

In [ ]:
def convert_load_flow_data(data, mem_vars):
    lf = {v: data[i][0] for i,v in enumerate(mem_vars)}
    for bus in (1,2,3):
        for x in 'dq':
            lf[f'G{bus}:{x}'] = lf.pop(f'g{bus}:{x}')
            lf[f'bus{bus}:{x}'] = lf[f'G{bus}:{x}']
            
    solution = {key: {} for key in ('generators','buses','loads')}

    Ptot, Qtot = 0, 0
    for gen in range(1,4):
        key = f'G{gen}:'
        V = lf[key + 'd'] + 1j * lf[key + 'q']
        I = lf[key + 'id'] + 1j * lf[key + 'iq']
        S = V * I.conj()
        P = -S.real * 1e-6
        Q = -S.imag * 1e-6
        solution['generators'][f'G{gen}'] = {
            'P': P,
            'Q': Q,
            'I': abs(I) * 1e-3 / np.sqrt(3),
            'V': abs(V) * 1e-3 / np.sqrt(3),
            'Vl': abs(V) * 1e-3
        }
        Ptot += P
        Qtot += Q
    solution['generators']['Ptot'], solution['generators']['Qtot'] = Ptot, Qtot

    for bus in range(1,10):
        key = f'bus{bus}:'
        V = lf[key + 'd'] + 1j * lf[key + 'q']
        Vl = abs(V) * 1e-3
        solution['buses'][f'Bus {bus}'] = {
            'voltage': Vl / VRATING[f'bus{bus}'],
            'V': Vl / np.sqrt(3),
            'Vl': Vl
        }

    for bus,load in zip((5,6,8),'ABC'):
        key = f'bus{bus}:'
        V = lf[key + 'd'] + 1j * lf[key + 'q']
        Vl = abs(V) * 1e-3
        solution['loads'][f'Load {load}'] = {
            'P': LOADS[load]['P'],
            'Q': LOADS[load]['Q'],
            'V': Vl / np.sqrt(3),
            'Vl': Vl
        }
        load = solution['loads'][f'Load {load}']
        load['I'] = np.sqrt(load['P'] ** 2 + load['Q'] ** 2) / load['Vl'] / np.sqrt(3)
    solution['loads']['Ptot'] = np.sum([load['P'] for load in LOADS.values()])
    solution['loads']['Qtot'] = np.sum([load['Q'] for load in LOADS.values()])
    
    return solution

This function prints the results of a load flow analysis

In [ ]:
def print_load_flow(results):
    print('\n===== Generators =====')
    for name,data in results['generators'].items():
        if name not in ('Ptot','Qtot'):
            print(f'{name}: P = {data["P"]:7.2f} MW, Q = {data["Q"]:6.2f} MVAR, ' + 
                  f'I = {data["I"]:6.3f} kA, V = {data["V"]:6.3f} kV.')
    print(f'Total P = {results["generators"]["Ptot"]:6.2f} MW, ' + 
          f'total Q = {results["generators"]["Qtot"]:6.2f} MVAR')

    print('\n======= Buses ========')
    for name,data in results['buses'].items():
        print(f'{name}: voltage = {data["voltage"]:5.3f} pu, V = {data["Vl"]:7.3f} kV.')
        
    print('\n======= Loads ========')
    for name,data in results['loads'].items():
        if name not in ('Ptot','Qtot'):
            print(f'{name}: P = {data["P"]:7.2f} MW, Q = {data["Q"]:6.2f} MVAR, ' + 
                  f'I = {data["I"]:6.3f} kA, V = {data["V"]:8.3f} kV.')
    print(f'Total P = {results["loads"]["Ptot"]:6.2f} MW, ' +
          f'total Q = {results["loads"]["Qtot"]:6.2f} MVAR')

#### Load the netlist in pan

The netlist is stored in the [wscc.pan](wscc.pan) file: it is possible to overload the network by varying the parameter `LAMBDA` in the interval [0,1]: this has the effect of increasing by `LAMBDA`% both the power generated by two generators (the slack generator is untouched) and the power (active and reactive) absorbed by the three loads.

Notice that at this point no analysis or simulation is performed.

In [ ]:
netlist_file = 'wscc.pan'
ok,libs = pan.load_netlist(netlist_file)
if not ok:
    raise Exception('load_netlist failed.')

#### Define which variables to save in memory
In this way we won't have to load the data from file. The voltage at each node in a netlist can be saved in memory and accessed via Python's `pan.ui` functions.

In [ ]:
mem_vars = []
for bus in range(1, 10):
    if bus < 4:
        mem_vars.append(f'g{bus}:d')
        mem_vars.append(f'g{bus}:q')
    else:
        mem_vars.append(f'bus{bus}:d')
        mem_vars.append(f'bus{bus}:q')
for gen in range(1, 4):
    mem_vars.append(f'G{gen}:id')
    mem_vars.append(f'G{gen}:iq')

#### Run the load flow

Here we run a batch of load flow analyses, changing the parameter `LAMBDA` with the `pan.alter` command. In order for this to work, the `LAMBDA` parameter must be tagged as "run-time" in the netlist. This is accomplished by the line

`Al_dummy_lambda alter param="LAMBDA" rt=yes`

in the [wscc.pan](wscc.pan) file.

In [ ]:
LAMBDA = np.r_[0 : 31] / 100
LF = []
for l in LAMBDA:
    pan.alter('Allam', 'LAMBDA', l, libs, annotate=1)
    data = pan.DC('Lf', mem_vars=mem_vars, libs=libs, nettype=1, print='yes')
    LOADS = {
        'A': {'P': 125. * (1+l), 'Q': 50. * (1+l)},
        'B': {'P':  90. * (1+l), 'Q': 30. * (1+l)},
        'C': {'P': 100. * (1+l), 'Q': 35. * (1+l)}
    }
    LF.append(convert_load_flow_data(data, mem_vars))

#### Load the results obtained with PowerFactory, if the file is available

In [ ]:
try:
    LF_PowerFactory = pickle.load(open('WSCC_9_bus_load_flow_PowerFactory_overload.pkl', 'rb'))
    print('Successfully loaded the file with PowerFactory results.')
except:
    LF_PowerFactory = None
    print('Cannot find the file with PowerFactory results.')

#### Plot the results
We plot active and reactive power of the slack generator as a function of the overload parameter `LAMBDA` and compare the results obtained with pan with those obtained with PowerFactory.

In [ ]:
P = np.array([lf['generators']['G1']['P'] for lf in LF])
Q = np.array([lf['generators']['G1']['Q'] for lf in LF])
if LF_PowerFactory is not None:
    LAMBDA_PowerFactory = np.array(list(LF_PowerFactory.keys()))
    P_PowerFactory = np.array([lf['generators']['G1']['P'] for lf in LF_PowerFactory.values()])
    Q_PowerFactory = np.array([lf['generators']['G1']['Q'] for lf in LF_PowerFactory.values()])

fig = plt.figure(figsize=(8.5/2.54, 5/2.54))
ax = plt.axes([0.15, 0.2, 0.8, 0.75])
if LF_PowerFactory is not None:
    ax.plot(LAMBDA_PowerFactory[0::10], P_PowerFactory[0::10], 'ks', markersize=8)
    ax.plot(LAMBDA_PowerFactory[0::10], Q_PowerFactory[0::10], 'rs', markersize=8)
ax.plot(LAMBDA, P, 'ko-', label='P', lw=1, markerfacecolor='w', markeredgewidth=1, markersize=4)
ax.plot(LAMBDA, Q, 'ro-', label='Q', lw=1, markerfacecolor='w', markeredgewidth=1, markersize=4)
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel(r'$\lambda$', fontsize=8)
ax.set_ylabel('Power [MW]', fontsize=8)
ax.legend(loc='lower right')
ax.grid(which='major', axis='y', linewidth=0.5, linestyle=':')
ax.set_ylim([0,100])
ax.tick_params(axis='x', labelsize=8)
ax.tick_params(axis='y', labelsize=8)
fig.savefig('WSCC_load_flow.pdf')

#### Save the results to disk

In [ ]:
data = {'LAMBDA': LAMBDA, 'P': P, 'Q': Q}
if LF_PowerFactory is not None:
    data['LAMBDA_PowerFactory'] = LAMBDA_PowerFactory
    data['P_PowerFactory'] = P_PowerFactory
    data['Q_PowerFactory'] = Q_PowerFactory
pickle.dump(data, open('WSCC_load_flow.pkl', 'wb'))